<h1> SET-UP </h1>

<h3> DEFINITIONS </h3>

In [12]:
import json
import requests

RULES_URL = 'https://api.rules.art/graphql'

<h3> FUNCTIONS </h3>

In [13]:
def group_by_season(card_list):
    # Grouping by season
    cards = {}
    for card in card_list:
        if 'cardModel' in card:
            season = card['cardModel']['season']
        else:
            season = card['season']
        if season not in cards:
            cards[season] = []
        cards[season].append(card)

    return cards

In [14]:
def all_cards_model():
    query = f"""
    {{
        allCardModels{{
            slug,
            uid,
            scarcity{{
                name
            }},
            season,
            artistName
        }}
    }}
    """

    r = requests.post(RULES_URL, json={'query': query})
    #print(r.status_code)

    try:
        r_data = r.json()
        rules_data = r_data.get('data', {}).get('allCardModels')
        return group_by_season(rules_data)
    except ValueError:
        raise ValueError("Error: Invalid JSON response from the API.")

    

In [15]:
def owned_cards(account_name):
  
  query = f"""
  {{
    user(slug: "{account_name}") {{
      ownedCardModels {{
        cardModel {{
          slug,
          uid,
          scarcity {{
            name
          }}
          artistName,
          season
        }},
        serialNumbers
      }}
    }}
  }}
  """
  r = requests.post(RULES_URL, json={'query': query})
  #print(r.status_code)

  try:
    r_data = r.json()
    user_data = r_data.get('data', {}).get('user', {}).get('ownedCardModels')
    return group_by_season(user_data)
  except ValueError:
      raise ValueError("Error: Invalid JSON response from the API.")
  
  


In [16]:
def all_owned_cards(card_list):
    seen_slugs = set()
    doublons = {}
    # Find serialNumbers[] with more than 1 value
    for card in card_list:
        slug = card['cardModel']['slug']
        serial_numbers = card['serialNumbers']
        if len(serial_numbers) > 1:
            doublons[slug] = serial_numbers
        else:
            seen_slugs.add(slug)
    return doublons

In [17]:
def find_doublons(card_list):
  seen_slugs = set()
  doublons = {}
  # Find serialNumbers[] with more than 1 value
  for card in card_list:
      slug = card['cardModel']['slug']
      serial_numbers = card['serialNumbers']
      if len(serial_numbers) > 1:
          doublons[slug] = serial_numbers
      else:
          seen_slugs.add(slug)
  return doublons

In [18]:
def print_good_format(card_list):

    # Printing the data in the desired format

    for season, slugs in card_list.items():
        print(f"SEASON {season}:")
        for slug in slugs:
            if 'cardModel' in slug:
                serial_numbers = slug['serialNumbers']
                artist_name = slug['cardModel']['artistName']
                scarcity_name = slug['cardModel']['scarcity']['name']
                uid = slug['cardModel']['uid']
                print(f"{uid} - {artist_name} - {scarcity_name} - #{serial_numbers}")
            else:
                artist_name = slug['artistName']
                scarcity_name = slug['scarcity']['name']
                uid = slug['uid']
                print(f"{uid} - {artist_name} - {scarcity_name}")
        print()  # Empty line between seasons

***

<h1> RULES - ALL CARDS </h1>

In [19]:
all_cards = all_cards_model()
#print(f"{all_cards}\n")
print_good_format(all_cards)

SEASON 1:
1 - Le Règlement - common
2 - Le Règlement - platinium
3 - Caballero - common
4 - Caballero - platinium
5 - JeanJass - common
6 - JeanJass - platinium
7 - Zuukou Mayzie - common
8 - Zuukou Mayzie - platinium
9 - Benjamin Epps - common
10 - Benjamin Epps - platinium
11 - thaHomey - common
12 - thaHomey - platinium
13 - So La Lune - common
14 - So La Lune - platinium
15 - Sheldon - common
16 - Sheldon - platinium
17 - Youv Dee - common
18 - Youv Dee - platinium
19 - Livaï - common
20 - Livaï - platinium
21 - Flem - common
22 - Flem - platinium
23 - Spider Zed - common
24 - Spider Zed - platinium
25 - Lycos - common
26 - Lycos - platinium
27 - Winnterzuko - common
28 - Winnterzuko - platinium
29 - Jewel Usain - common
30 - Jewel Usain - platinium
31 - Slimka - common
32 - Slimka - platinium
33 - 8ruki - common
34 - 8ruki - platinium
35 - Mahdi Ba - common
36 - Mahdi Ba - platinium
37 - Jwles - common
38 - Jwles - platinium
39 - Azur - common
40 - Azur - platinium
41 - Moji x Sbo

***

<h1> MAIN ACCOUNT ({{main_account}}) </h1>

In [20]:
main_account = input("What is your main account profile name ? ").casefold().replace(" ", "")
card_list_main_account = owned_cards(main_account)
print(card_list_main_account)

{1: [{'cardModel': {'slug': 'zinee-season-1-halloween', 'uid': 75, 'scarcity': {'name': 'halloween'}, 'artistName': 'Zinée', 'season': 1}, 'serialNumbers': [1842]}, {'cardModel': {'slug': 'jeanjass-season-1-halloween', 'uid': 69, 'scarcity': {'name': 'halloween'}, 'artistName': 'JeanJass', 'season': 1}, 'serialNumbers': [2081]}, {'cardModel': {'slug': 'caballero-season-1-halloween', 'uid': 68, 'scarcity': {'name': 'halloween'}, 'artistName': 'Caballero', 'season': 1}, 'serialNumbers': [1763]}, {'cardModel': {'slug': 'flem-season-1-common', 'uid': 21, 'scarcity': {'name': 'common'}, 'artistName': 'Flem', 'season': 1}, 'serialNumbers': [2808, 3107, 3464, 3873, 3883]}, {'cardModel': {'slug': 'le-reglement-season-1-common', 'uid': 1, 'scarcity': {'name': 'common'}, 'artistName': 'Le Règlement', 'season': 1}, 'serialNumbers': [1928, 3827]}, {'cardModel': {'slug': 'lycos-season-1-common', 'uid': 25, 'scarcity': {'name': 'common'}, 'artistName': 'Lycos', 'season': 1}, 'serialNumbers': [3051, 

<h3> ALL CARDS OWNED </h3>

In [21]:
print_good_format(card_list_main_account)

SEASON 1:
75 - Zinée - halloween - #[1842]
69 - JeanJass - halloween - #[2081]
68 - Caballero - halloween - #[1763]
21 - Flem - common - #[2808, 3107, 3464, 3873, 3883]
1 - Le Règlement - common - #[1928, 3827]
25 - Lycos - common - #[3051, 3385]
58 - Doums - common - #[1938, 2627]
43 - Rowjay - common - #[1998, 3770]
23 - Spider Zed - common - #[3287]
53 - JMK$ - common - #[3391]
30 - Jewel Usain - platinium - #[261]
31 - Slimka - common - #[3274]
29 - Jewel Usain - common - #[3227]
9 - Benjamin Epps - common - #[3623]
15 - Sheldon - common - #[2923, 3129]
39 - Azur - common - #[3614]
62 - Ysos - common - #[2309, 2437]
47 - Skuna - common - #[3682]
27 - Winnterzuko - common - #[3622]
60 - Bigflo - common - #[2579]
33 - 8ruki - common - #[3966]
51 - Di-Meh - common - #[3873]
41 - Moji x Sboy - common - #[3718]
49 - Deen Burbigo - common - #[3926]
56 - Kerchak - common - #[3101]
17 - Youv Dee - common - #[4067]
45 - Zinée - common - #[4034]
35 - Mahdi Ba - common - #[4119]
11 - thaHomey

<h3> DOUBLONS </h3>

In [22]:
doublons = find_doublons(card_list_main_account)


print(F"Doublons ({main_account}):")
for slug, serial_numbers in doublons.items():
    print(f"{slug}: {serial_numbers}")

#print(json.dumps(card_list_main_account, indent=2))

TypeError: 'int' object is not subscriptable

***

<h1> SECOND ACCOUNT ({{second_account}}) </h1> 

In [ ]:
second_account = input("What is your second account profile name ? ").casefold().replace(" ", "")
card_list_second_account = execute_query(second_account)

AttributeError: 'NoneType' object has no attribute 'get'

<h3> ALL CARDS OWNED </h3>

In [ ]:
#print(card_list_main_account)

cards = all_owned_cards(card_list_second_account)

print(F"All cards owned ({second_account}):")
for slug, serial_numbers in cards.items():
    print(f"{slug}: {serial_numbers}")

All cards owned (foguete):
zinee-season-1-halloween: [1842]
jeanjass-season-1-halloween: [2081]
caballero-season-1-halloween: [1763]
flem-season-1-common: [2808, 3107, 3464, 3873, 3883]
le-reglement-season-1-common: [1928, 3827]
lycos-season-1-common: [3051, 3385]
doums-season-1-common: [1938, 2627]
rowjay-season-1-common: [1998, 3770]
spider-zed-season-1-common: [3287]
jmks-season-1-common: [3391]
jewel-usain-season-1-platinium: [261]
slimka-season-1-common: [3274]
jewel-usain-season-1-common: [3227]
benjamin-epps-season-1-common: [3623]
sheldon-season-1-common: [2923, 3129]
azur-season-1-common: [3614]
ysos-season-1-common: [2309, 2437]
skuna-season-1-common: [3682]
winnterzuko-season-1-common: [3622]
bigflo-season-1-common: [2579]
8ruki-season-1-common: [3966]
di-meh-season-1-common: [3873]
moji-x-sboy-season-1-common: [3718]
deen-burbigo-season-1-common: [3926]
kerchak-season-1-common: [3101]
youv-dee-season-1-common: [4067]
zinee-season-1-common: [4034]
mahdi-ba-season-1-common: [

<h3> DOUBLONS </h3>

In [ ]:
doublons = find_doublons(card_list_second_account)
#print(doublons)

print(F"Doublons ({second_account}):")
for slug, serial_numbers in doublons.items():
    print(f"{slug}: {serial_numbers}")

#print(json.dumps(card_list_main_account, indent=2))

Doublons (foguete):
flem-season-1-common: [2808, 3107, 3464, 3873, 3883]
le-reglement-season-1-common: [1928, 3827]
lycos-season-1-common: [3051, 3385]
doums-season-1-common: [1938, 2627]
rowjay-season-1-common: [1998, 3770]
sheldon-season-1-common: [2923, 3129]
ysos-season-1-common: [2309, 2437]
houdi-season-2-holo: [176, 256]
